In [13]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 20.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.5
    Uninstalling langsmith-0.1.5:
      Successfully uninstalled langsmith-0.1.5
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.25
    Uninstalling langchain-core-0.1.25:
      Successfully uninstalled langchain-core-0.1.25
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.21
    Uninstalling langchain-community-0.0.21:
      Successfully uninstalled langchain-community-0.0.21
  Attempting uninstall

In [14]:
from huggingface_hub import hf_hub_download
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat

In [15]:
model_type = "BioMistral/BioMistral-7B-GGUF"
model_file = "ggml-model-Q5_K_M.gguf"
local_path = "./models/" + model_type

In [16]:
local_llm = hf_hub_download(
    repo_id = model_type,
    filename = model_file,
    local_dir=local_path
)

In [17]:
llm = LlamaCpp(
    model_path=local_llm
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ./models/BioMistral/BioMistral-7B-GGUF/ggml-model-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llam

In [18]:
model = Llama2Chat(llm=llm)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)

In [11]:
conversation = [
    {"role": "user", "content": "Hi"},
]

In [ ]:
model.